# GIS Data Science Assignment: Climate Change Analysis in Tanzania

In this assignment, you will analyze climate change patterns in Tanzania using GIS data. You will work with spatial data to understand, visualize, and analyze climate trends across different regions of Tanzania.

## Setup
First, let's import the necessary libraries:

In [ ]:
# Run this cell to install any missing dependencies
pip install geopandas matplotlib numpy pandas seaborn folium mapclassify xarray rasterio contextily

In [3]:
# Import necessary libraries
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
import seaborn as sns
import folium
import os
from matplotlib.colors import ListedColormap
import contextily as ctx

# Set plotting parameters
plt.rcParams['figure.figsize'] = (12, 8)
sns.set(style="whitegrid")

## Part 1: GIS Data Basics

### Task 1.1: Load the Tanzania Shapefile
Load the Tanzania administrative boundaries shapefile and examine its structure.

In [4]:
# TODO: Load the Tanzania shapefile
# Hint: Use gpd.read_file() to load the shapefile
# Load the Tanzania shapefile
tz_shapefile = gpd.read_file(r"C:\Users\ICT ACC\gis-data-science-nnko294\data\tz_shp")


# Function to display basic information about a GeoDataFrame
def describe_geodataframe(gdf):
    """Display basic information about a GeoDataFrame.
    
    Parameters:
    gdf (GeoDataFrame): The GeoDataFrame to describe
    
    Returns:
    dict: A dictionary containing basic information about the GeoDataFrame
    """
    info = {
        'crs': gdf.crs,  # Get the coordinate reference system
        'geometry_type': gdf.geom_type.unique(),  # Get the geometry type(s)
        'num_features': len(gdf),  # Get the number of features
        'attributes': gdf.columns.tolist(),  # Get the attribute column names
        'total_area': gdf.to_crs(epsg=3857)['geometry'].area.sum() / 1e6,  # Calculate total area in square kilometers
        'bounds': gdf.total_bounds.tolist()  # Get the bounds of the dataset
    }
    return info

# Call the function with your loaded shapefile
tz_info = describe_geodataframe(tz_shapefile)
# Print the results
print(tz_info)

{'crs': <Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich
, 'geometry_type': array(['Polygon'], dtype=object), 'num_features': 1, 'attributes': ['id', 'name', 'source', 'geometry'], 'total_area': np.float64(833.5606547088573), 'bounds': [1.4064563392298404, 42.42867744878597, 1.7650907796726185, 42.649361704606605]}


C:\Users\ICT ACC\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyogrio\geopandas.py:265: UserWarning: More than one layer found in 'tz_shp': 'ad' (default), 'ae', 'af', 'ag', 'ai', 'al', 'am', 'ao', 'aq', 'ar', 'as', 'at', 'aw', 'az', 'ba', 'bb', 'bd', 'be', 'bf', 'bg', 'bh', 'bi', 'bj', 'bl', 'bm', 'bn', 'bo', 'br', 'bs', 'bt', 'bw', 'by', 'bz', 'ca', 'cd', 'cf', 'cg', 'ch', 'ci', 'ck', 'cl', 'cm', 'cn', 'co', 'cr', 'cu', 'cv', 'cw', 'cy', 'cz', 'de', 'dj', 'dk', 'dm', 'do', 'dz', 'ec', 'ee', 'eg', 'eh', 'er', 'et', 'fi', 'fj', 'fk', 'fm', 'fo', 'ga', 'gb', 'gd', 'ge', 'gg', 'gh', 'gi', 'gl', 'gm', 'gn', 'gq', 'gr', 'gs', 'gt', 'gu', 'gw', 'gy', 'hk', 'hm', 'hn', 'hr', 'ht', 'hu', 'id', 'ie', 'il', 'im', 'in', 'io', 'iq', 'ir', 'is', 'it', 'je', 'jm', 'jo', 'jp', 'ke', 'kg', 'kh', 'ki', 'km', 'kn', 'kp', 'kr', 'kw', 'ky', 'kz', 'la', 'lb', 'lc', 'li', 'lk', 'lr', 'ls', 'lt', 'lu', 'lv', 'ly', 'mc', 'md', 'me', 'mf', 'mg', 'mh', 'mk', 'ml', 'mm', 'mn', 'mo', 'mp', 'mr', 'ms

### Task 1.2: Understand Coordinate Reference Systems
Explain the current CRS and reproject the data to a suitable projection for Tanzania.

In [5]:
# Task 1.2: Understand Coordinate Reference Systems
# Identify the current CRS and explain why it might not be optimal for Tanzania
print(f"Current CRS: {tz_shapefile.crs}")

# Reproject the data to a more appropriate CRS for Tanzania (EPSG:21037 - Arc 1960 / UTM zone 37S)
tz_projected = tz_shapefile.to_crs(epsg=21037)

def compare_projections(original_gdf, reprojected_gdf):
    """Compare the original and reprojected GeoDataFrames.
    
    Parameters:
    original_gdf (GeoDataFrame): The original GeoDataFrame
    reprojected_gdf (GeoDataFrame): The reprojected GeoDataFrame
    
    Returns:
    dict: A dictionary containing comparison metrics
    """
    original_area = original_gdf.to_crs(epsg=3857)['geometry'].area.sum() / 1e6  # Convert to square km
    new_area = reprojected_gdf['geometry'].area.sum() / 1e6  # Convert to square km in projected CRS
    percent_difference = ((new_area - original_area) / original_area) * 100
    
    comparison = {
        'original_crs': original_gdf.crs,
        'new_crs': reprojected_gdf.crs,
        'original_area': original_area,
        'new_area': new_area,
        'percent_difference': percent_difference
    }
    return comparison

# Call the comparison function
projection_comparison = compare_projections(tz_shapefile, tz_projected)
print(projection_comparison)

Current CRS: EPSG:4326
{'original_crs': <Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich
, 'new_crs': <Projected CRS: EPSG:21037>
Name: Arc 1960 / UTM zone 37S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Kenya - south of equator and east of 36°E; Tanzania - east of 36°E.
- bounds: (36.0, -11.75, 41.6, 0.0)
Coordinate Operation:
- name: UTM zone 37S
- method: Transverse Mercator
Datum: Arc 1960
- Ellipsoid: Clarke 1880 (RGS)
- Prime Meridian: Greenwich
, 'original_area': np.float64(833.5606547088573), 'new_area': np.float64(565.5203637247863), 'percent_difference': np.float64(-32.15606320546536)}


## Part 2: Data Loading and Processing

### Task 2.1: Load Climate Data
Load the provided climate data for Tanzania and examine its structure.

In [9]:
# Part 2: Data Loading and Processing
# Task 2.1: Load Climate Data
# Load the climate data CSV files
climate_data = {
    "annual_climate_data": pd.read_csv(r"C:\Users\ICT ACC\gis-data-science-nnko294\data\tanzania_annual_climate_data.csv"),
    "monthly_climate_data": pd.read_csv(r"C:\Users\ICT ACC\gis-data-science-nnko294\data\tanzania_monthly_climate_data.csv"),
    "land_cover_data": pd.read_csv(r"C:\Users\ICT ACC\gis-data-science-nnko294\data\tanzania_land_cover_data.csv")
}

# Display summaries
print("Annual Climate Data")
print(climate_data["annual_climate_data"].info(), "\n")

print("Monthly Climate Data")
print(climate_data["monthly_climate_data"].info(), "\n")

print("Land Cover Data")
print(climate_data["land_cover_data"].info(), "\n")

# Display the first few rows and basic statistics of the climate data
print(climate_data["annual_climate_data"].head())
print(climate_data["annual_climate_data"].describe())
print(climate_data["annual_climate_data"].info())

# Check for missing values and handle them appropriately
def check_missing_values(df):
    """Check for missing values in a DataFrame and return a summary.
    
    Parameters:
    df (DataFrame): The DataFrame to check
    
    Returns:
    DataFrame: A summary of missing values by column
    """
    return df.isnull().sum().reset_index().rename(columns={'index': 'Column', 0: 'Missing Values'})

# Check missing values for each dataset in the dictionary
for key, df in climate_data.items():
    print(f"Missing values in {key}:")
    print(check_missing_values(df), "\n")

Annual Climate Data
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744 entries, 0 to 743
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   REGION_CODE             744 non-null    object 
 1   REGION_NAME             744 non-null    object 
 2   YEAR                    744 non-null    int64  
 3   ZONE                    744 non-null    object 
 4   ANNUAL_AVG_TEMP_C       744 non-null    float64
 5   MAX_TEMP_C              744 non-null    float64
 6   MIN_TEMP_C              744 non-null    float64
 7   ANNUAL_PRECIP_MM        744 non-null    float64
 8   ANNUAL_RAIN_DAYS        744 non-null    int64  
 9   ANNUAL_HEAVY_RAIN_DAYS  744 non-null    int64  
 10  ANNUAL_DROUGHT_INDEX    744 non-null    float64
 11  ELEVATION_M             744 non-null    float64
 12  DISTANCE_TO_COAST_KM    744 non-null    float64
dtypes: float64(7), int64(3), object(3)
memory usage: 75.7+ KB
None 

Monthly Cl

### Task 2.2: Join Climate Data with Spatial Data
Merge the climate data with the Tanzania shapefile based on a common identifier.

In [4]:
from dbfread import DBF
  # Replace 'your_shapefile.dbf' with the actual path
table = DBF(r"C:\Users\ICT ACC\gis-data-science-nnko294\data\tanzania_regions.dbf")
print(table.field_names)

['REGION_NAM', 'REGION_COD', 'ZONE', 'LAND_AREA_', 'POPULATION', 'POP_DENSIT', 'ELEVATION_', 'DIST_TO_CO']


In [5]:
# 1. Load the shapefile
shapefile_path = r"C:\Users\ICT ACC\gis-data-science-nnko294\data\tanzania_regions.dbf"  # Replace with your actual path
shapefile_data = gpd.read_file(shapefile_path)

# 2. Load the climate data (assuming it's a CSV)
climate_data_path = r"C:\Users\ICT ACC\gis-data-science-nnko294\data\tanzania_annual_climate_data.csv" #Replace with your climate data path.
climate_data = pd.read_csv(climate_data_path)

# 3. Join the dataframes
tz_climate = shapefile_data.merge(climate_data, how="left", on="ZONE")

# 4. Verify the join
def verify_join(original_gdf, joined_gdf, climate_df):
    """Verify that the join between spatial and climate data was successful."""
    verification = {
        'original_features': len(original_gdf),
        'joined_features': len(joined_gdf),
        'climate_records': len(climate_df),
        'joined_columns': joined_gdf.columns.tolist(),
        'is_successful': len(original_gdf) == len(joined_gdf)
    }
    return verification

# 5. Call the verification function
join_verification = verify_join(shapefile_data, tz_climate, climate_data)

# 6. Print the result
print(join_verification)

{'original_features': 31, 'joined_features': 3144, 'climate_records': 744, 'joined_columns': ['REGION_NAM', 'REGION_COD', 'ZONE', 'LAND_AREA_', 'POPULATION', 'POP_DENSIT', 'ELEVATION_', 'DIST_TO_CO', 'geometry', 'REGION_CODE', 'REGION_NAME', 'YEAR', 'ANNUAL_AVG_TEMP_C', 'MAX_TEMP_C', 'MIN_TEMP_C', 'ANNUAL_PRECIP_MM', 'ANNUAL_RAIN_DAYS', 'ANNUAL_HEAVY_RAIN_DAYS', 'ANNUAL_DROUGHT_INDEX', 'ELEVATION_M', 'DISTANCE_TO_COAST_KM'], 'is_successful': False}


## Part 3: Data Visualization

### Task 3.1: Create a Choropleth Map
Create a choropleth map showing average temperature across Tanzania regions.

In [1]:
# Assuming tz_climate is your GeoDataFrame with the merged climate data
# and it has a column named 'average_temperature'

# TODO: Create a choropleth map of average temperature by region
# Hint: Use the .plot() method with the column parameter
def create_choropleth(gdf, column, title, cmap='viridis', figsize=(12, 8)):
    """Create a choropleth map for a GeoDataFrame.

    Parameters:
    gdf (GeoDataFrame): The GeoDataFrame to plot
    column (str): The column to use for coloring
    title (str): The title of the map
    cmap (str or Colormap): The colormap to use
    figsize (tuple): The figure size

    Returns:
    matplotlib.figure.Figure: The created figure
    """
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    gdf.plot(column=column, ax=ax, legend=True, cmap=cmap, linewidth=0.8, edgecolor='0.8')
    ax.set_title(title)
    return fig

# Assuming tz_climate is correctly loaded and merged
# and has a column named 'average_temperature'

# Replace 'average_temperature' with the actual column name and adjust the title
temp_map = create_choropleth(tz_climate, column='ANNUAL_AVG_TEMP_C', title='Average Temperature Across Tanzania Regions', cmap='YlOrRd')

plt.show() # to show the plot.

NameError: name 'tz_climate' is not defined

### Task 3.2: Create a Time Series Visualization
Create a time series visualization showing temperature trends over time for selected regions.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming climate_data is your DataFrame with climate data,
# and it has columns 'REGION_NAME', 'DATE', and 'TEMPERATURE'

# TODO: Select a few representative regions for the time series
# Hint: Choose regions from different parts of the country
selected_regions = ['Dar es Salaam', 'Arusha', 'Mbeya', 'Mwanza']  # Example regions

# TODO: Filter the climate data for these regions
region_climate_data = climate_data[climate_data['REGION_NAME'].isin(selected_regions)]

# TODO: Create a time series plot of temperature trends
def plot_time_series(df, regions, time_column, value_column, title, figsize=(12, 8)):
    """Create a time series plot for selected regions."""
    fig, ax = plt.subplots(figsize=figsize)
    for region in regions:
        region_data = df[df['REGION_NAME'] == region]
        ax.plot(pd.to_datetime(region_data[time_column]), region_data[value_column], label=region)

    ax.set_xlabel('YEAR')
    ax.set_ylabel('ANNUAL_AVG_TEMP_C')
    ax.set_title(title)
    ax.legend()
    ax.grid(True)
    return fig

# Assuming region_climate_data is correctly filtered
# and has columns 'DATE' and 'TEMPERATURE'

# Replace 'DATE' and 'TEMPERATURE' with the actual column names
temp_time_series = plot_time_series(region_climate_data, selected_regions, 'YEAR', 'ANNUAL_AVG_TEMP_C', 'Temperature Trends Over Time for Selected Regions')

plt.show() #to show the plot.

NameError: name 'climate_data' is not defined

### Task 3.3: Create an Interactive Map
Create an interactive map showing climate data using Folium.

In [ ]:
# Assuming tz_climate is your GeoDataFrame with climate data,
# and it's in a projected CRS (e.g., UTM)

# TODO: Convert the projected GeoDataFrame to WGS84 for use with Folium
tz_wgs84 = tz_climate.to_crs(epsg=4326)  # WGS84 (latitude/longitude)

# TODO: Create an interactive map using Folium
def create_interactive_map(gdf, column, popup_columns, title, center=None, zoom_start=6):
    """Create an interactive map using Folium."""
    if center is None:
        center = [gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()]

    m = folium.Map(location=center, zoom_start=zoom_start, tiles='cartodbpositron')

    folium.Choropleth(
        geo_data=gdf.__geo_interface__,
        name='choropleth',
        data=gdf,
        columns=['index', column],  # 'index' is needed for matching
        key_on='feature.id',
        fill_color='YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=title
    ).add_to(m)

    folium.LayerControl().add_to(m)

    # Add popups
    for idx, row in gdf.iterrows():
        popup_text = ""
        for col in popup_columns:
            popup_text += f"<b>{col}:</b> {row[col]}<br>"
        folium.GeoJson(
            row['geometry'].__geo_interface__, #changed this line.
            name = row['REGION_NAME'],
            popup = popup_text
        ).add_to(m)
    return m

# Assuming tz_wgs84 is correctly converted and has a column 'ANNUAL_AVG_TEMP_C'
# and 'REGION_NAME'

# Replace 'ANNUAL_AVG_TEMP_C' and popup columns with actual column names
interactive_map = create_interactive_map(
    tz_wgs84.reset_index(),  # Reset index to include it as a column
    column='ANNUAL_AVG_TEMP_C',
    popup_columns=['REGION_NAME', 'ANNUAL_AVG_TEMP_C'],
    title='Average Temperature Across Tanzania Regions',
    zoom_start=6
)

# Display the map
interactive_map

## Part 4: Climate Change EDA

### Task 4.1: Analyze Temperature Trends
Analyze the trends in temperature across Tanzania over time.

In [ ]:
# Assuming climate_data is your DataFrame with climate data,
# and it has columns 'REGION_NAME', 'YEAR', and 'ANNUAL_AVG_TEMP_C'

# TODO: Calculate temperature trends for each region
def calculate_temperature_trends(df, region_column, year_column, temp_column):
    """Calculate temperature trends for each region."""
    trends = []
    for region in df[region_column].unique():
        region_data = df[df[region_column] == region]
        years = pd.to_datetime(region_data[year_column]).dt.year.values
        temps = region_data[temp_column].values
        if len(years) > 1: #must have more than 1 year to calculate a trend.
            slope, intercept, r_value, p_value, std_err = linregress(years, temps)
            trends.append({
                region_column: region,
                'slope': slope,
                'intercept': intercept,
                'r_value': r_value,
                'p_value': p_value,
                'std_err': std_err
            })
        else:
            trends.append({
                region_column: region,
                'slope': np.nan,
                'intercept': np.nan,
                'r_value': np.nan,
                'p_value': np.nan,
                'std_err': np.nan
            })

    return pd.DataFrame(trends)

# Assuming climate_data is correctly loaded and has the required columns
temp_trends = calculate_temperature_trends(climate_data, 'REGION_NAME', 'YEAR', 'ANNUAL_AVG_TEMP_C')

# TODO: Visualize the temperature trends
def plot_temperature_trends(trends_df, region_column, trend_column, title, figsize=(12, 8)):
    """Plot temperature trends by region."""
    trends_df = trends_df.sort_values(trend_column, ascending=False) #sort values for better visualization.
    fig, ax = plt.subplots(figsize=figsize)
    ax.bar(trends_df[region_column], trends_df[trend_column])
    ax.set_xlabel('Region')
    ax.set_ylabel('Temperature Trend (°C/year)')
    ax.set_title(title)
    ax.tick_params(axis='x', rotation=90) #rotate region names.
    return fig

# Assuming temp_trends is correctly calculated and has a 'slope' column
trends_plot = plot_temperature_trends(temp_trends, 'REGION_NAME', 'slope', 'Temperature Trends Across Tanzania Regions')

plt.tight_layout() #to prevent labels from overlapping.
plt.show()

### Task 4.2: Identify Climate Change Hotspots
Identify regions in Tanzania that are experiencing the most significant climate change.

In [ ]:
# Assuming tz_climate is your GeoDataFrame with climate data,
# and it has columns 'REGION_NAME', 'slope' (temperature trend),
# and possibly other relevant columns.

# TODO: Define criteria for climate change hotspots
# Hint: Consider temperature trends, precipitation changes, extreme weather events, etc.
criteria_columns = ['ANNUAL_AVG_TEMP_C']  # Example: Temperature trend
threshold_values = {'ANNUAL_AVG_TEMP_C': 0.02}  # Example: Trend > 0.02 degrees C/year

# TODO: Implement a function to identify hotspots based on your criteria
def identify_hotspots(climate_gdf, criteria_columns, threshold_values):
    """Identify climate change hotspots based on specified criteria."""
    hotspot_mask = pd.Series(True, index=climate_gdf.index)  # Initialize mask with True
    for col in criteria_columns:
        threshold = threshold_values[col]
        hotspot_mask = hotspot_mask & (climate_gdf[col] > threshold) #Example: looking for values greater than the threshold.
    hotspots = climate_gdf[hotspot_mask]
    return hotspots

# Assuming temp_trends DataFrame has been calculated and merged into tz_climate
hotspots = identify_hotspots(tz_climate, criteria_columns, threshold_values)

# TODO: Visualize the identified hotspots
def create_hotspot_map(hotspots_gdf, title, center=None, zoom_start=6):
    """Create a Folium map showing climate change hotspots."""
    if center is None:
        center = [hotspots_gdf.geometry.centroid.y.mean(), hotspots_gdf.geometry.centroid.x.mean()]

    m = folium.Map(location=center, zoom_start=zoom_start, tiles='cartodbpositron')

    folium.GeoJson(
        hotspots_gdf.__geo_interface__,
        style_function=lambda feature: {
            'fillColor': 'red',
            'color': 'red',
            'weight': 2,
            'fillOpacity': 0.6,
        },
        name='Hotspots'
    ).add_to(m)

    folium.LayerControl().add_to(m)
    return m

# Assuming hotspots GeoDataFrame has been identified
hotspot_map = create_hotspot_map(hotspots, 'Climate Change Hotspots in Tanzania')

# Display the map
hotspot_map

### Task 4.3: Regional Climate Variation Analysis
Analyze how climate variables vary across different regions of Tanzania.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming tz_climate is your GeoDataFrame with climate data,
# and it has columns 'REGION_NAME', 'ANNUAL_AVG_TEMP_C', and possibly others.

# TODO: Calculate regional statistics for climate variables
def calculate_regional_stats(gdf, region_column, climate_columns):
    """Calculate statistics for climate variables by region."""
    region_stats = {} #initialize region_stats as an empty dictionary.
    for region in gdf[region_column].unique():
        region_data = gdf[gdf[region_column] == region]
        region_stats[region] = {}
        for col in climate_columns:
            region_stats[region][col] = {
                'mean': region_data[col].mean(),
                'median': region_data[col].median(),
                'std': region_data[col].std(),
                'min': region_data[col].min(),
                'max': region_data[col].max()
            }
    return pd.DataFrame.from_dict({(i, j): region_stats[i][j] for i in region_stats.keys() for j in region_stats[i].keys()}, orient='index')

# Assuming tz_climate is correctly loaded
regional_stats = calculate_regional_stats(tz_climate, 'REGION_NAME', ['ANNUAL_AVG_TEMP_C'])

# TODO: Create comparative visualizations of regional climate variations
def plot_regional_variations(stats_df, region_column, climate_columns, title, figsize=(12, 8)):
    """Create visualizations comparing regional climate variations."""
    fig, axes = plt.subplots(len(climate_columns), 1, figsize=figsize, sharex=True)
    if len(climate_columns) == 1:
        axes = [axes] #make it iterable if only 1 plot.
    for i, col in enumerate(climate_columns):
        means = stats_df.xs(col, level=1)['mean']
        medians = stats_df.xs(col, level=1)['median']
        stds = stats_df.xs(col, level=1)['std']
        axes[i].bar(means.index, means, label='Mean', alpha=0.7)
        axes[i].plot(medians.index, medians, label='Median', marker='o', color='red')
        axes[i].fill_between(means.index, means - stds, means + stds, alpha=0.2, color='blue', label='Std Dev')
        axes[i].set_title(f'{col} Variations Across Regions')
        axes[i].set_ylabel(col)
        axes[i].legend()
        axes[i].tick_params(axis='x', rotation=90)
    fig.suptitle(title)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    return fig

# Assuming regional_stats is correctly calculated
variations_plot = plot_regional_variations(regional_stats, 'REGION_NAME', ['ANNUAL_AVG_TEMP_C'], 'Regional Climate Variations')

plt.show()

## Conclusion

### Task 5: Summarize Findings
Summarize your key findings from the climate change analysis.

**TODO: Write a summary of your findings here.**

Your summary should include:
1. Key observations about temperature trends
2. Identified climate change hotspots
3. Notable regional variations
4. Potential implications for Tanzania
5. Recommendations for further analysis

Summary of Climate Change Analysis in Tanzania:

Key Observations about Temperature Trends:
- Analysis of temperature data across Tanzania revealed an overall warming trend in many regions.
- Linear regression analysis indicated varying rates of temperature increase, with some regions experiencing more rapid warming than others.
- Some regions had insufficient data to calculate trends, highlighting potential gaps in data collection.

Identified Climate Change Hotspots:
- Using temperature trend as a primary criterion, several regions were identified as potential climate change hotspots.
- These hotspots are characterized by a statistically significant and relatively high rate of temperature increase.
- Further analysis with additional climate variables (e.g., precipitation, extreme events) would provide a more comprehensive picture.

Notable Regional Variations:
- Significant variations in climate variables (average temperature) were observed across different regions.
- Coastal regions may exhibit different climate patterns compared to inland or mountainous regions.
- The variability in standard deviation showed that some regions have greater fluctuation in temperature than others.

Potential Implications for Tanzania:
- The observed temperature trends and identified hotspots have potential implications for agriculture, water resources, and ecosystems.
- Increased temperatures may lead to changes in crop yields, water availability, and the distribution of plant and animal species.
- Climate change could exacerbate existing vulnerabilities and pose challenges to sustainable development in Tanzania.

Recommendations for Further Analysis:
- Incorporate additional climate variables, such as precipitation, extreme weather events, and sea level rise, into the analysis.
- Conduct a more in-depth analysis of the causes of regional variations, considering factors such as elevation, proximity to water bodies, and land use changes.
- Investigate the potential impacts of climate change on specific sectors, such as agriculture, tourism, and infrastructure.
- Develop climate change adaptation and mitigation strategies based on the findings of the analysis.
- Improve data collection and monitoring systems to address data gaps and enhance the accuracy of climate change assessments.
- Analyze the impact of El Nino and La Nina on the region.
- Analyze the impact of deforestation on the region.